In [1]:

# Imports
import os
from predicting_nails.params import *
import matplotlib.pyplot as plt
#from tensorflow.keras import layers, models, Sequential
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications import ResNet50, EfficientNetB0
#from keras.preprocessing.image import ImageDataGenerator
from predicting_nails.prediction.get_data import *
from PIL import Image

In [ ]:
# Plots Loss and accuracy of the train veresus the validation data
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label = 'train' + exp_name)
    ax1.plot(history.history['val_loss'], label = 'val' + exp_name)
    ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

In [ ]:
# Initialize model Baseline for a CNN model

def initialize_baseline_model():
      '''instanciate and return the CNN architecture'''
      model = models.Sequential()

      model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train_processed.shape[1:4]))
      model.add(layers.MaxPool2D(pool_size=(5,5)))
      model.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
      model.add(layers.MaxPool2D(pool_size=(2,2)))
      model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))
      model.add(layers.MaxPool2D(pool_size=(2,2)))
      model.add(layers.Flatten())
      model.add(layers.Dense(32, activation='relu'))
      model.add(layers.Dense(1, activation='sigmoid'))

      return model

In [ ]:
# Compile the model as a binary classification model

def compile_model(model):
    '''return a compiled model for the baseline'''
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
# Set the first layers to be untrainable

def set_nontrainable_layers(model):
    model.trainable = False
    return model

In [ ]:
# Initialize and compile a model for a transfer learning

def build_model(base_model):
      '''instanciate and return the CNN architecture'''
      base_model = set_nontrainable_layers(base_model)
      flattening_layer = layers.Flatten()
      dense_layer = layers.Dense(500, activation='relu')
      prediction_layer = layers.Dense(1, activation='sigmoid')

      model = Sequential([
      base_model,
      flattening_layer,
      dense_layer,
      prediction_layer
      ])
      return compile_model(model)

In [2]:
# Get data
if os.path.exists(LOCAL_DATA_PATH):
    X, y = load_simple_data(LOCAL_DATA_PATH)


In [ ]:
# split data in test and train and preprocess data
X_train_processed = 0
X_test_processed = 0

# CNN baseline model

In [ ]:
# Fit the baseline model on the train data
es = EarlyStopping(patience = 10)
baseline_model = compile_model(initialize_baseline_model())

history_baseline = baseline_model.fit(
  X_train_processed,
  y_train,
  validation_split = 0.2,
  epochs = 500,
  batch_size = 32,
  verbose = 1,
  callbacks = [es]
)

In [ ]:
# Plot Loss and accuracy of the basline model
plot_history(history_baseline, title='Baseline', axs=None, exp_name="")

# VGG16 model

In [ ]:
# Loads the VGG16 model
def load_VGG16_model():
    model = VGG16(weights="imagenet", include_top=False, input_shape=X_train_processed.shape[1:4].shape)
    return model

In [ ]:
# Fit the VGG16 model on the train data
es = EarlyStopping(patience = 10)
VGG16_model = build_model(load_VGG16_model())

history_VGG16 = VGG16_model.fit(
  X_train_processed,
  y_train,
  validation_split = 0.2,
  epochs = 500,
  batch_size = 32,
  verbose = 1,
  callbacks = [es]
)

In [ ]:
# Plot Loss and accuracy of the VGG16 model
plot_history(history_VGG16, title='VGG16', axs=None, exp_name="")

# ResNet50

In [ ]:
# Loads the ResNet50 model
def load_ResNet50_model():
    model = ResNet50(weights="imagenet", include_top=False, input_shape=X_train_processed.shape[1:4].shape)
    return model

In [ ]:
# Fit the ResNet50 model on the train data
es = EarlyStopping(patience = 10)
ResNet50_model = build_model(load_ResNet50_model())

history_ResNet50 = ResNet50_model.fit(
  X_train_processed,
  y_train,
  validation_split = 0.2,
  epochs = 500,
  batch_size = 32,
  verbose = 1,
  callbacks = [es]
)

In [ ]:
# Plot Loss and accuracy of the ResNet50 model
plot_history(history_ResNet50, title='ResNet50', axs=None, exp_name="")

# EfficientNetB0

In [ ]:
# Loads the EfficientNetB0 model
def load_EfficientNetB0_model():
    model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=X_train_processed.shape[1:4].shape)
    return model

In [ ]:
# Fit the EfficientNetB0 model on the train data
es = EarlyStopping(patience = 10)
EfficientNetB0_model = build_model(load_EfficientNetB0_model())

history_EfficientNetB0 = EfficientNetB0_model.fit(
  X_train_processed,
  y_train,
  validation_split = 0.2,
  epochs = 500,
  batch_size = 32,
  verbose = 1,
  callbacks = [es]
)

In [ ]:
# Plot Loss and accuracy of the EfficientNetB0 model
plot_history(history_EfficientNetB0, title='EfficientNetB0', axs=None, exp_name="")

# Data augmentation

In [ ]:
# Add Data Augmentation if your model is overfitting
datagen = ImageDataGenerator(
    featurewise_center = False,
    featurewise_std_normalization = False,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    brightness_range = (0.5, 1.),
    zoom_range = (0.3, 1.5))


# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_processed)

model_data_aug = build_model()

train_flow = datagen.flow(X_train_processed, y_train, batch_size=16)
val_flow = datagen.flow(X_val_preprocessed, y_val, batch_size=16)

es = EarlyStopping(monitor = 'val_accuracy',
                   mode = 'max',
                   patience = 5,
                   verbose = 1,
                   restore_best_weights = True)

history_data_aug = model_data_aug.fit(train_flow,
                                      validation_data = val_flow,
                                      epochs = 50,
                                      callbacks = [es])